Si planeamos desplegar este tipo de modelos ante todo debemos conocer como realizar dos tareas fundamentales:

* Evaluar sus resultados
* Monitorizar o trazar sus acciones

Dependiendo del framework de implementación esto puede resultar algo más complejo aunque la mayoría incluyen integraciones abiertas con OpenTelemetry o trazabilidad propia como es el caso e LangChain vía LangSmith, una plataforma nube que gestiona las trazas que el framework instrumenta de forma sencilla.

# LangSmith

[LangSmith](https://www.langchain.com/langsmith) se presenta como un entorno nube pero también una vía para instrumentar de forma sencilla nuestras interacciones dentro del contexto de uso de las LLMs. Simplemente incluyendo las variables de entorno:

```
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=<TOKEN>
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_PROJECT=<PROYECTO>
```

Podemos hacer un seguimiento de las llamadas, cuellos de botella y respuestas arrojadas por el proveedor que estemos empleando.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

Esto es suficiente para que nuestras acciones queden completamente registradas.

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Create the LLM Chain using LangChain
prompt = PromptTemplate(
    input_variables=["input"],
    template="Traduce el siguiente texto al Francés: {input}"
)
chain = prompt | llm

# Generate the translations
translation = chain.invoke("Hello, how are you?")
print(translation.content)

La traducción más común y general de "Hello, how are you?" al francés es:

**Bonjour, comment allez-vous ?**

Aquí hay algunas alternativas, dependiendo del contexto y la formalidad:

*   **Salut, comment vas-tu ?** (Más informal, para amigos o conocidos)
*   **Bonjour, ça va ?** (Muy común, informal)
*   **Bonsoir, comment allez-vous ?** (Si es por la tarde/noche)

La primera opción, **Bonjour, comment allez-vous ?**, es la más segura para cualquier situación.


![langsmith](../assets/images/langsmithpro.png)

Veremos que otros frameworks como [Agno](https://docs.agno.com/examples/concepts/observability/langsmith-via-openinference) nos permiten volcar la información también a LangSmith aunque resulta algo menos sencillo instrumentar las aplicaciones en este caso.

# Observabilidad

Otro aspecto clave es poder evaluar las respuestas de nuestro modelo. ¿Presenta sesgos? ¿Vuelca información sensible? ¿Sus respuestas son correctas? Podemos acotar mucho con un buen trabajo de prompting pero nunca estamos seguros cuando se trata de modelos probabilísticos.

La gran pega es que debemos evaluar dos textos, el texto que sabemos es una buena respuesta y el arrojado por el modelo... de ahí que una de las modalidades más extendidas se trate de usar una LLM como juez (dándole las instrucciones adecuadas). Así es como operan soluciones como:

* [Deepeval](https://deepeval.com/)
* [Openevals](https://github.com/langchain-ai/openevals)

Veamos algunos ejemplos aunque podéis encontrar más y más específicos en la documentación de [LangSmith](https://docs.smith.langchain.com/evaluation/tutorials).

Por un lado tenemos por ejemplo, la relevancia de la respuesta:

$$
\text{Answer relevancy} = \frac{\text{Number of relevant statements}}{\text{Total number of statements}}
$$

[AnswerRelevancy](https://deepeval.com/docs/metrics-answer-relevancy#how-is-it-calculated)


In [3]:
import os
from deepeval.models import GeminiModel
from deepeval.metrics import AnswerRelevancyMetric

model = GeminiModel(
    model_name="gemini-2.0-flash",
    api_key=os.environ.get("GOOGLE_API_KEY"),
    temperature=0
)

answer_relevancy = AnswerRelevancyMetric(model=model, verbose_mode=True)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

pregunta = "Tengo tos persistente y fiebre. ¿Debería preocuparme?"
response = llm.invoke(pregunta)
print(response.content)

Sí, definitivamente deberías preocuparte si tienes tos persistente y fiebre. Estos son síntomas comunes de varias enfermedades, algunas de las cuales pueden ser graves.

**Aquí te explico por qué debes preocuparte y qué deberías hacer:**

**Posibles causas:**

*   **Infecciones respiratorias:**
    *   **Gripe (Influenza):** Causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.
    *   **Resfriado común:**  Generalmente más leve que la gripe, pero puede causar tos, congestión nasal, dolor de garganta y fiebre baja.
    *   **Bronquitis:** Inflamación de los bronquios, causa tos (a menudo con mucosidad), sibilancias y dificultad para respirar.
    *   **Neumonía:** Infección de los pulmones, puede ser causada por bacterias, virus u hongos.  Causa tos (a menudo con mucosidad), fiebre alta, dificultad para respirar y dolor en el pecho.
    *   **COVID-19:**  Causa una amplia gama de síntomas, incluyendo fiebre, tos, fatiga, dolor de garganta, pérdida del gusto u olfato y difi

Montemos ahora el caso de test. ¿Qué respuesta entendemos sería correcta?

In [5]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=pregunta,
    actual_output=response.content,
    expected_output="Una tos y fiebre persistentes podrían indicar una variedad de enfermedades, desde una infección viral leve hasta afecciones más graves como neumonía o COVID-19. Debe buscar atención médica si sus síntomas empeoran, persisten durante más de unos días o están acompañados de dificultad para respirar, dolor en el pecho u otros signos preocupantes."
)

answer_relevancy.measure(test_case)

Output()

**************************************************

Answer Relevancy Verbose Logs

**************************************************

Statements:
[
    "Sí, definitivamente deberías preocuparte si tienes tos persistente y fiebre.",
    "Estos son síntomas comunes de varias enfermedades.",
    "Algunas de las enfermedades pueden ser graves.",
    "Infecciones respiratorias can be a possible cause.",
    "Gripe (Influenza) can be a possible cause.",
    "Gripe causes fiebre, tos, dolor de garganta, dolores musculares y fatiga.",
    "Resfriado común can be a possible cause.",
    "Resfriado común is generalmente más leve que la gripe.",
    "Resfriado común can causa tos, congestión nasal, dolor de garganta y fiebre baja.",
    "Bronquitis can be a possible cause.",
    "Bronquitis is an inflamación de los bronquios.",
    "Bronquitis causa tos, sibilancias y dificultad para respirar.",
    "Neumonía can be a possible cause.",
    "Neumonía is an infección de los pulmones.",
    "Neumonía can be caused by bacterias, virus u hongos.",
    "Neumonía causa tos, fiebre alta, dificultad para respirar y dolor en el pecho.",
    "COVID-19 can be a possible cause.",
    "COVID-19 causa una amplia gama de síntomas, incluyendo fiebre, tos, fatiga, dolor de garganta, pérdida del 
gusto u olfato y dificultad para respirar.",
    "Virus sincitial respiratorio (VSR) can be a possible cause.",
    "Virus sincitial respiratorio (VSR) is común en niños pequeños.",
    "Virus sincitial respiratorio (VSR) can afectar a adultos, especialmente aquellos con sistemas inmunitarios 
debilitados.",
    "Asma can be a possible cause.",
    "Asma generalmente no causa fiebre.",
    "Una infección respiratoria puede desencadenar un ataque de asma con tos y dificultad para respirar.",
    "Alergias can be a possible cause.",
    "Las alergias estacionales pueden causar tos, pero generalmente no fiebre.",
    "Enfermedad pulmonar obstructiva crónica (EPOC) can be a possible cause.",
    "Una exacerbación de la EPOC puede causar tos y dificultad para respirar.",
    "Tos ferina (Pertussis) can be a possible cause.",
    "Tos ferina is an infección bacteriana altamente contagiosa.",
    "Tos ferina causa tos severa e incontrolable.",
    "Un médico puede determinar la causa de tu tos y fiebre.",
    "El tratamiento dependerá de la causa subyacente.",
    "La neumonía bacteriana requiere antibióticos.",
    "La gripe puede requerir antivirales.",
    "Algunas infecciones respiratorias pueden llevar a complicaciones graves.",
    "Complications can be neumonía, dificultad respiratoria, sepsis o incluso la muerte.",
    "Si tienes una enfermedad contagiosa como la gripe o COVID-19, el diagnóstico temprano puede ayudarte a tomar 
medidas para evitar propagarla a otras personas.",
    "Consulta a un médico lo antes posible.",
    "Puedes programar una cita con tu médico de cabecera o acudir a una clínica de atención urgente.",
    "Si tienes dificultad para respirar o dolor en el pecho, busca atención médica de emergencia.",
    "Sé específico sobre tus síntomas.",
    "Informa a tu médico sobre cualquier condición médica que tengas.",
    "Pregunta a tu médico si necesitas pruebas como un hisopo nasal para detectar gripe o COVID-19, una radiografía
de tórax o análisis de sangre.",
    "Toma los medicamentos según las indicaciones y sigue las recomendaciones de tu médico sobre el descanso, la 
hidratación y el aislamiento.",
    "Quédate en casa y evita el contacto cercano con otras personas hasta que te sientas mejor y tu médico te 
indique que puedes volver a tus actividades normales.",
    "El descanso ayuda a tu cuerpo a combatir la infección.",
    "Bebe mucho líquido, como agua, jugo o caldo.",
    "Un humidificador puede ayudar a aliviar la tos y la congestión nasal.",
    "Los medicamentos de venta libre como el paracetamol o el ibuprofeno pueden ayudar a reducir la fiebre y el 
dolor.",
    "Los descongestionantes pueden ayudar a aliviar la congestión nasal.",
    "Consulta a tu médico o farmacéutico antes de tomar cualquier medicamento de venta libre, especialmente si 
tienes otras condi

======================================================================

1.0

Existen [multitud de métricas](https://deepeval.com/docs/metrics-introduction) que podemos emplear basadas en este mismo principio.

In [6]:
from deepeval import evaluate
from deepeval.metrics import (
    BiasMetric,
    PIILeakageMetric
)

# Sesgos
bias_metric = BiasMetric(threshold=0.5, model=model)

# Filtrado de información sensible o privada
pii_metric = PIILeakageMetric(threshold=0.0, model=model)

evaluate(test_cases=[test_case], metrics=[bias_metric, pii_metric])

✨ You're running DeepEval's latest Bias Metric! (using gemini-2.0-flash, strict=False, async_mode=True)...

✨ You're running DeepEval's latest PII Leakage Metric! (using gemini-2.0-flash, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Bias (score: 0.0, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.00 because there are no identified reasons for bias, suggesting the output is unbiased and commendable., error: None)
  - ✅ PII Leakage (score: 1.0, threshold: 0.0, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 1.00 because no privacy violations were identified, indicating the highest level of privacy preservation., error: None)

For test case:

  - input: Tengo tos persistente y fiebre. ¿Debería preocuparme?
  - actual output: Sí, definitivamente deberías preocuparte si tienes tos persistente y fiebre. Estos son síntomas comunes de varias enfermedades, algunas de las cuales pueden ser graves.

**Aquí te explico por qué debes preocuparte y qué deberías hacer:**

**Posibles causas:**

*   **Infecciones respiratorias:**
    *   **Gripe (Influenza):** Causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.
    *   **Resfr

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Bias', threshold=0.5, success=True, score=0.0, reason='The score is 0.00 because there are no identified reasons for bias, suggesting the output is unbiased and commendable.', strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Opinions:\n[] \n \nVerdicts:\n[]'), MetricData(name='PII Leakage', threshold=0.0, success=True, score=1.0, reason='The score is 1.00 because no privacy violations were identified, indicating the highest level of privacy preservation.', strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Extracted PII:\n[] \n \nVerdicts:\n[]')], conversational=False, multimodal=False, input='Tengo tos persistente y fiebre. ¿Debería preocuparme?', actual_output='Sí, definitivamente deberías preocuparte si tienes tos persistente y fiebre. Estos son síntomas comunes de varias e

Podemos incluir incluso lo correcto de la respuesta, instruir a la LLM juez cómo queremos que evalúe el resultado.

In [7]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determina cómo de acertada es la respuesta dada la respuesta correcta indicada. Evalúa longitud de la respuesta de forma que ser conciso sea mejor.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=model
)

evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.6, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The actual output is accurate and provides a comprehensive overview of potential causes and actions to take. However, it is significantly longer and more detailed than the expected output, including redundant information. The expected output is more concise while still addressing the core concern., error: None)

For test case:

  - input: Tengo tos persistente y fiebre. ¿Debería preocuparme?
  - actual output: Sí, definitivamente deberías preocuparte si tienes tos persistente y fiebre. Estos son síntomas comunes de varias enfermedades, algunas de las cuales pueden ser graves.

**Aquí te explico por qué debes preocuparte y qué deberías hacer:**

**Posibles causas:**

*   **Infecciones respiratorias:**
    *   **Gripe (Influenza):** Causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.
    *   **Resfriado común:**  Generalmente más leve que la 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=True, score=0.6, reason='The actual output is accurate and provides a comprehensive overview of potential causes and actions to take. However, it is significantly longer and more detailed than the expected output, including redundant information. The expected output is more concise while still addressing the core concern.', strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\nDetermina cómo de acertada es la respuesta dada la respuesta correcta indicada. Evalúa longitud de la respuesta de forma que ser conciso sea mejor. \n \nEvaluation Steps:\n[\n    "1. Compare the Actual Output to the Expected Output to determine accuracy. A more accurate output is better.",\n    "2. Assess the length of the Actual Output. Shorter, more concise outputs are preferred.",\n    "3. Evaluate if 